This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/intro.html).  I recommend having both windows open, side-by-side!

## Instructions for running this notebook on Deepnote

- Log in (the free account will be sufficient for this class)
- "Duplicate" the document.  Icon is in the top right next to Login. 
- Run all of the cells (can use the "Run notebook" icon just above this cell)
- If a new "MeshCat" window does not open automatically, then click on the url printed just below "StartMeshcat" in the second code cell of the notebook.
- In meshcat, click "Open controls" and use the sliders that appear to teleop.
- When you're done, click the "Stop Simulation" button in meshcat.  This will move you to the second example...

In [ ]:
import numpy as np
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.geometry import MeshcatVisualizer, StartMeshcat
from pydrake.manipulation.planner import (
    DifferentialInverseKinematicsIntegrator,
    DifferentialInverseKinematicsParameters)
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import MeshcatPoseSliders, WsgButton


In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Teleop Example (2D)

In this example, we assemble a diagram with all of the relevant subsystems (the manipulation station, the meshcat visualizer, and some systems that provide a minimal teleop interface and convert the teleop output from end-effector commands into joint commands.  We'll learn more about each of these components in the following chapters.

**NOTE:** If you command the robot to move its gripper beyond what is possible, then you get a message about "differential IK" failing.  I've left that in for now (rather than setting very conservative slider limits) partly because it has tutorial value.  We'll understand it more precisely soon!  For now, just stop the simulation and rerun the cell if you get stuck.


In [ ]:
def teleop_2d():
  builder = DiagramBuilder()

  station = builder.AddSystem(ManipulationStation())
  station.SetupPlanarIiwaStation()
  station.AddManipulandFromFile(
      "drake/examples/manipulation_station/models/"
      + "061_foam_brick.sdf",
      RigidTransform(RotationMatrix.Identity(), [0.6, 0, 0]))
  # TODO(russt): Add planar joint to brick
  station.Finalize()

  visualizer = MeshcatVisualizer.AddToBuilder(
      builder, station.GetOutputPort("query_object"), meshcat)
  meshcat.Set2dRenderMode()

  robot = station.get_controller_plant()
  params = DifferentialInverseKinematicsParameters(
      robot.num_positions(), robot.num_velocities())

  time_step = 0.005
  params.set_timestep(time_step)
  iiwa14_velocity_limits = np.array([1.4, 1.3, 2.3])
  params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                    iiwa14_velocity_limits))
  # These constants are in body frame.
  params.set_end_effector_velocity_gain([.1, 0, 0, 0, .1, .1])
  differential_ik = builder.AddSystem(
      DifferentialInverseKinematicsIntegrator(
          robot, robot.GetFrameByName("iiwa_link_7"), time_step, params))
  builder.Connect(differential_ik.get_output_port(),
                  station.GetInputPort("iiwa_position"))

  meshcat.DeleteAddedControls()
  teleop = builder.AddSystem(MeshcatPoseSliders(meshcat,
      min_range=MeshcatPoseSliders.MinRange(
          roll=0, x=-0.6, z=0.0),
      max_range=MeshcatPoseSliders.MaxRange(
          roll=2*np.pi, x=0.8, z=1.1),
      value=MeshcatPoseSliders.Value(pitch=0, yaw=0, y=0),
      visible=MeshcatPoseSliders.Visible(pitch=False, yaw=False, y=False) 
  ))
  builder.Connect(teleop.get_output_port(0), 
                  differential_ik.get_input_port())
  wsg_teleop = builder.AddSystem(WsgButton(meshcat))
  builder.Connect(wsg_teleop.get_output_port(0),
                  station.GetInputPort("wsg_position"))

  diagram = builder.Build()
  simulator = Simulator(diagram)
  context = simulator.get_mutable_context()
  station_context = station.GetMyMutableContextFromRoot(context)

  q0 = station.GetOutputPort("iiwa_position_measured").Eval(
      station_context)
  differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
  diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
  differential_ik.SetPositions(diff_ik_context, q0)
  teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

  if running_as_notebook:  # Then we're not just running as a test on CI.
      simulator.set_target_realtime_rate(1.0)

      meshcat.AddButton("Stop Simulation")
      while meshcat.GetButtonClicks("Stop Simulation") < 1:
          simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
      meshcat.DeleteButton("Stop Simulation")

  else:
      simulator.AdvanceTo(0.1)

teleop_2d()


# Teleop Example (3D)

The physics and geometry engines running in the simulation above are actually running in 3D.  This example is almost identical, but we'll use the (default) 3D visualization and add more sliders for controlling the full `roll`, `pitch`, `yaw` angles and `x`, `y`, `z` positions of the end effector.

In [ ]:
def teleop_3d():
    builder = DiagramBuilder()

    station = builder.AddSystem(ManipulationStation())

    station.SetupClutterClearingStation()
    #ycb_objects = CreateClutterClearingYcbObjectList()
    #for model_file, X_WObject in ycb_objects:
    #    station.AddManipulandFromFile(model_file, X_WObject)
    station.AddManipulandFromFile(
        "drake/examples/manipulation_station/models/"
        + "061_foam_brick.sdf",
        RigidTransform(RotationMatrix.Identity(), [0, -0.6, 0.2]))
    station.Finalize()

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)
    meshcat.ResetRenderMode()

    robot = station.get_controller_plant()
    params = DifferentialInverseKinematicsParameters(
        robot.num_positions(), robot.num_velocities())

    time_step = 0.005
    params.set_timestep(time_step)
    # True velocity limits for the IIWA14 (in rad, rounded down to the first
    # decimal)
    iiwa14_velocity_limits = np.array([1.4, 1.4, 1.7, 1.3, 2.2, 2.3, 2.3])
    params.set_joint_velocity_limits((-iiwa14_velocity_limits,
                                    iiwa14_velocity_limits))
    params.set_end_effector_velocity_gain([.1]*6)
    differential_ik = builder.AddSystem(DifferentialInverseKinematicsIntegrator(
            robot, robot.GetFrameByName("iiwa_link_7"), time_step, params))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa_position"))

    meshcat.DeleteAddedControls()
    teleop = builder.AddSystem(MeshcatPoseSliders(meshcat,
        min_range = MeshcatPoseSliders.MinRange(
            roll=0, pitch=-0.5, yaw=-np.pi, x=-0.6, y=-0.8, z=0.0),
        max_range = MeshcatPoseSliders.MaxRange(
            roll=2*np.pi, pitch=np.pi, yaw=np.pi, x=0.8, y=0.3, z=1.1)
    ))
    builder.Connect(teleop.get_output_port(0), 
                    differential_ik.get_input_port())
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0),
                    station.GetInputPort("wsg_position"))

    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()

    station_context = station.GetMyMutableContextFromRoot(context)

    q0 = station.GetOutputPort("iiwa_position_measured").Eval(
        station_context)
    differential_ik.get_mutable_parameters().set_nominal_joint_position(q0)
    diff_ik_context = differential_ik.GetMyMutableContextFromRoot(context)
    differential_ik.SetPositions(diff_ik_context, q0)
    teleop.SetPose(differential_ik.ForwardKinematics(diff_ik_context))

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)

        meshcat.AddButton("Stop Simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        meshcat.DeleteButton("Stop Simulation")
    
    else:
        simulator.AdvanceTo(0.1)

teleop_3d()